In [21]:
%load_ext autoreload
%autoreload 2

import os
os.environ['TORCH_HOME'] = "/disks/sdb/torch_home"

import logging
logging.getLogger().setLevel(logging.WARNING)

from awesome_glue.task import *
from awesome_glue.config import Config

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [124]:
config = Config()
task = Task(config)
task.from_pretrained()
config

cache for AGNEWS-spacy.data exists
*** load AGNEWS-spacy.data from cache cost 15.2 seconds
cache for AGNEWS-glove.vec exists
*** load AGNEWS-glove.vec from cache cost 0.228 seconds


Basic Args:
	--task_id=AGNEWS
	--finetunable=True
	--arch=lstm
	--pretrain=glove
	--_model_name=
	--mode=attack
	--adv_data=nogit/AGNEWS-lstm.hotflip.adv.bt.tsv
	--transform=identity
	--aug_data=
	--attack_method=pwws
	--attack_vectors=counter
	--attack_data_split=dev
	--attack_size=400
	--attack_gen_aug=False
	--attack_gen_adv=False
	--alchemist=False
	--seed=2
Deduced Args:
	--model_name=AGNEWS-lstm
	--tokenizer=spacy

In [18]:
scores = {}
stops = stopwords.words("english")
all_words = list(task.vocab.get_index_to_token_vocabulary().values())[2:]
for word in all_words:
    if "'" in word:
        continue
    scores[word] = round(task.predictor.predict(word)['probs'][0], 2)

In [130]:
# from awesome_glue.utils import FreqUtil
# FreqUtil.split_by_frequency(task.vocab, 19, 20)
task.vocab._retained_counter['tokens']['Microsoft']

6375

TypeError: 'Vocabulary' object is not subscriptable

In [75]:
total = 0
for i in range(30):
    sent = allenutil.as_sentence(task.dev_data[i])
    before = task.predictor.predict(sent)['probs'][0]
    if before > 0.5:
        sent = "above above " + sent
    else:
        sent = "too too " + sent
    after = task.predictor.predict(sent)['probs'][0]
    if (before - 0.5) * (after - 0.5) < 0:
        total += 1
print(total)

11


In [27]:
sorted(scores.items(), key=lambda x:x[1], reverse=False)[:5000][-1]

('voices', 0.13)